In [13]:
# CELDA 1: Preparación del entorno (comprobación/descarga segura de recursos NLTK)
import nltk

def asegurar_recurso(recurso_path, nombre_paquete):
    """
    Comprueba si existe el recurso en nltk.data; si no, lo descarga.
    recurso_path: ruta usada por nltk.data.find (ej: 'tokenizers/punkt')
    nombre_paquete: nombre usado en nltk.download (ej: 'punkt')
    """
    import nltk.data
    try:
        nltk.data.find(recurso_path)
        # recurso ya disponible
        # print(f"Recurso '{nombre_paquete}' ya disponible.")
    except LookupError:
        print(f"Descargando recurso NLTK: {nombre_paquete} ...")
        nltk.download(nombre_paquete, quiet=False)

# Aseguramos tokenizador y tagger (se descargan solo si faltan)
asegurar_recurso('tokenizers/punkt', 'punkt')
asegurar_recurso('taggers/averaged_perceptron_tagger', 'averaged_perceptron_tagger')

print("✅ Recursos NLTK comprobados/descargados si fue necesario.")

✅ Recursos NLTK comprobados/descargados si fue necesario.


In [14]:
# CELDA 2: Importaciones
from nltk import word_tokenize, pos_tag
from pprint import pprint

print("✅ Importaciones completadas.")

✅ Importaciones completadas.


In [15]:
# CELDA 3: Implementación de analisis_morfologico (robusta)
def analisis_morfologico(oracion):
    """
    Realiza análisis morfológico (tokens, tipos, ratio tipo/token, POS tags).
    Implementación robusta: comprueba recursos NLTK, maneja errores y devuelve resultado parcial si hace falta.

    Args:
        oracion (str): oración en español (cadena)

    Returns:
        dict: {
            'total_tokens': int,
            'total_tipos': int,
            'ratio_tt': float (3 decimales),
            'pos_tags': [(token, etiqueta), ...]
        }
    """
    import nltk
    # Asegurar recursos por si se llamó a la función en otro entorno sin ejecutar la celda 1
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=False)
    try:
        nltk.data.find('taggers/averaged_perceptron_tagger')
    except LookupError:
        nltk.download('averaged_perceptron_tagger', quiet=False)

    resultado = {
        'total_tokens': 0,
        'total_tipos': 0,
        'ratio_tt': 0.0,
        'pos_tags': []
    }

    if not isinstance(oracion, str):
        raise ValueError("El argumento 'oracion' debe ser una cadena de texto.")

    try:
        # Tokenización (sin language param)
        tokens = word_tokenize(oracion)
    except Exception as e:
        # fallback: split simple si tokenización falla (no ideal, pero evita crash)
        print("⚠️ Error en word_tokenize():", str(e))
        print("Usando fallback con simple split() para tokens.")
        tokens = oracion.split()

    # Conteos
    total_tokens = len(tokens)
    total_tipos = len(set(tokens))
    ratio_tt = round(total_tipos / total_tokens, 3) if total_tokens > 0 else 0.0

    # Etiquetado POS (intentamos usar averaged_perceptron_tagger)
    try:
        pos_tags = pos_tag(tokens, tagset='universal')
    except Exception as e:
        # Si falla, devolvemos tokens sin etiquetas y avisamos
        print("⚠️ Error en pos_tag():", str(e))
        print("Devolviendo tokens sin etiquetas POS.")
        pos_tags = [(t, None) for t in tokens]

    resultado['total_tokens'] = total_tokens
    resultado['total_tipos'] = total_tipos
    resultado['ratio_tt'] = ratio_tt
    resultado['pos_tags'] = pos_tags

    return resultado

In [16]:
# CELDA 4: Pruebas con oraciones de ejemplo
oracion_ejemplo = "El gato negro salta alto y el perro corre rápido por el parque."

res = analisis_morfologico(oracion_ejemplo)
pprint(res)

# Pruebas adicionales
print("\nPrueba adicional 1:")
pprint(analisis_morfologico("Los estudiantes estudian programación en la universidad cada mañana."))

print("\nPrueba adicional 2:")
pprint(analisis_morfologico("Hoy llueve, pero mañana saldrá el sol."))

⚠️ Error en word_tokenize(): 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Usando fallback con simple split() para tokens.
⚠️ Error en pos_tag(): 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
 

In [17]:
# ============================================================
# 🔹 CELDA 5: Análisis breve de resultados
# ============================================================

"""
🔍 ANÁLISIS DE RESULTADOS:

1️⃣ Los valores de 'total_tokens' indican la cantidad de palabras (y signos) detectados por el tokenizador.
2️⃣ 'total_tipos' representa cuántas palabras diferentes hay en la oración.
3️⃣ El 'ratio_tt' (tipos/tokens) mide la diversidad léxica:
    - Valores cercanos a 1 indican una oración con muchas palabras únicas.
    - Valores más bajos indican repeticiones o vocabulario limitado.
4️⃣ Las etiquetas POS proporcionan la categoría gramatical de cada palabra (sustantivo, verbo, adjetivo, etc.).

En general, el análisis morfológico permite observar la estructura gramatical y la variedad del vocabulario de un texto.
"""

"\n🔍 ANÁLISIS DE RESULTADOS:\n\n1️⃣ Los valores de 'total_tokens' indican la cantidad de palabras (y signos) detectados por el tokenizador.\n2️⃣ 'total_tipos' representa cuántas palabras diferentes hay en la oración.\n3️⃣ El 'ratio_tt' (tipos/tokens) mide la diversidad léxica: \n    - Valores cercanos a 1 indican una oración con muchas palabras únicas.\n    - Valores más bajos indican repeticiones o vocabulario limitado.\n4️⃣ Las etiquetas POS proporcionan la categoría gramatical de cada palabra (sustantivo, verbo, adjetivo, etc.).\n\nEn general, el análisis morfológico permite observar la estructura gramatical y la variedad del vocabulario de un texto.\n"